In [0]:
pip install azure-eventhub

In [0]:
service_credential = dbutils.secrets.get(scope="databricksscopename",key="databricksservicekey")

spark.conf.set("fs.azure.account.auth.type.adlsx5u224gv3xgd2.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.adlsx5u224gv3xgd2.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.adlsx5u224gv3xgd2.dfs.core.windows.net", "5c962712-e111-48b0-b9af-41f03175e216")
spark.conf.set("fs.azure.account.oauth2.client.secret.adlsx5u224gv3xgd2.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.adlsx5u224gv3xgd2.dfs.core.windows.net", "https://login.microsoftonline.com/72f988bf-86f1-41af-91ab-2d7cd011db47/oauth2/token")

In [0]:
import pyspark.pandas as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *
from azure.eventhub import EventHubProducerClient, EventHubConsumerClient, EventData

In [0]:
json_file_path = 'abfss://data@adlsx5u224gv3xgd2.dfs.core.windows.net/sales_orders_stream.json'  # Replace with your JSON file path

# Read a json file using pandas
df = ps.read_json(json_file_path)
df = df.head(5)

display(df)
# Convert the CSV file into JSON
json_data = df.to_json(orient='records')

In [0]:
print(json_data)

In [0]:
event_hub_connection_string = 'Endpoint=sb://streamdata-2f774f-ns.servicebus.windows.net/;SharedAccessKeyName=rule;SharedAccessKey={SharedAccessKey}'  # Replace with your connection string
event_hub_name = 'test'



producer = EventHubProducerClient.from_connection_string(event_hub_connection_string, eventhub_name=event_hub_name)

# Publish the JSON as events
try:
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json_data))
    producer.send_batch(event_data_batch)
except Exception as e:
    print('Error occurred: ', str(e))
finally:
    producer.close()